In [ ]:
# ==========================================
# Cell 0 : 추가 패키지 설치
# ==========================================

# 설치를 했는데 셀 3번 7번에서 작동이 안 될 경우 터미널에서 skn17_final_env 커널이 연결되어 있는 상황에서 설치
!pip install audio_separator
!pip install -U bitsandbytes

In [2]:
# ==========================================
# Cell 1: 경로 설정 및 토큰 활성화
# ==========================================

from pathlib import Path
import os, sys
import json

# ---- 프로젝트 루트 ----
PROJECT_ROOT = Path("/workspace/baseball_pipeline")
os.chdir(PROJECT_ROOT)

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)

# ---- 데이터 디렉토리 ----
from src import DATA_DIR, FAISS_DIR, FISH_ROOT

INPUT_VIDEO_DIR = DATA_DIR / "input_videos"
STT_RAW_DIR = DATA_DIR / "stt_raw"
STT_SEG_DIR = DATA_DIR / "stt_segments"
LLM_OUT_DIR = DATA_DIR / "llm_outputs"
TTS_AUDIO_DIR = DATA_DIR / "tts_audio"
OUTPUT_VIDEO_DIR = DATA_DIR / "output_videos"
AUDIO_ROOT = DATA_DIR / "audio_separator"
FRAMES_ROOT = DATA_DIR / "frames"
SRC_ROOT = PROJECT_ROOT / "src"

if str(SRC_ROOT) not in sys.path:
    sys.path.append(str(SRC_ROOT))

# 디렉토리 생성
for d in (DATA_DIR, INPUT_VIDEO_DIR, STT_RAW_DIR, STT_SEG_DIR, 
          LLM_OUT_DIR, TTS_AUDIO_DIR, OUTPUT_VIDEO_DIR, AUDIO_ROOT, 
          FRAMES_ROOT, FAISS_DIR, SRC_ROOT):
    d.mkdir(parents=True, exist_ok=True)

print("\n✅ 디렉토리 생성 완료")

# ---- API 토큰 설정 ----
CLOVA_INVOKE_URL = ""
CLOVA_SECRET_KEY = ""

HF_TOKEN = ""
# OPENAI_API_KEY = "sk-proj-..."  # 필요시 입력

if HF_TOKEN and "xxx" not in HF_TOKEN:
    os.environ["HF_TOKEN"] = HF_TOKEN
    os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
    os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN

# if OPENAI_API_KEY:
#     os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ API 토큰 설정 완료\n")

PROJECT_ROOT: /workspace/baseball_pipeline

✅ 디렉토리 생성 완료
✅ API 토큰 설정 완료



In [3]:
from src.google_mp4_download import download_gdrive_video

# ====== 여기에 구글 드라이브 링크 입력 ======
gdrive_url = "https://drive.google.com/file/d/1kqN0IAerBH72RnoQwXzM-T-rrwoaMLDn/view?usp=sharing"
VIDEO_NAME = "체코_대한민국_wbc.mp4"

local_video_path = download_gdrive_video(gdrive_url, dest_name=VIDEO_NAME)
video_stem = Path(VIDEO_NAME).stem

print(f"\n✅ 영상 다운로드 완료")
print(f"  video_stem: {video_stem}")
print(f"  경로: {local_video_path}\n")

[GDRIVE] file_id: 1kqN0IAerBH72RnoQwXzM-T-rrwoaMLDn
[GDRIVE] url    : https://drive.google.com/uc?id=1kqN0IAerBH72RnoQwXzM-T-rrwoaMLDn
[GDRIVE] output : /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4


Downloading...
From (original): https://drive.google.com/uc?id=1kqN0IAerBH72RnoQwXzM-T-rrwoaMLDn
From (redirected): https://drive.google.com/uc?id=1kqN0IAerBH72RnoQwXzM-T-rrwoaMLDn&confirm=t&uuid=69f2a8f3-c11d-49e4-b630-cc7d149a2e9b
To: /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4
100%|██████████| 1.29G/1.29G [00:25<00:00, 50.1MB/s]


[GDRIVE] 다운로드 완료: /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4

✅ 영상 다운로드 완료
  video_stem: 체코_대한민국_wbc
  경로: /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/체코_대한민국_wbc.mp4



In [ ]:
# ==========================================
# Cell 3: audio_separator 음성 분리
# ==========================================
from src.audio_separator import separate_audio_sota

track_dict = separate_audio_sota(
    video_path=str(local_video_path),
    output_dir=str(AUDIO_ROOT),
    device="cuda"
)

vocals_path = track_dict["vocals"]
no_vocals_path = track_dict["no_vocals"]

print(f"\n✅ 음성 분리 완료! (SOTA Performance)")
print(f"  - 해설(Vocals)    : {vocals_path}")
print(f"  - 현장음(No Vocals): {no_vocals_path}\n")


In [ ]:
# ==========================================
# Cell 4: STT (Clova Speech API)
# ==========================================
from pathlib import Path
import json

from src.stt_pipeline import run_stt_pipeline
from src.stt_event_splitter import stt_json_to_event_sets

print(f"[STT] Clova STT 시작")
print(f"  입력: {vocals_path}")

STT_KEYWORD_XLSX = PROJECT_ROOT / "stt.xlsx"
if STT_KEYWORD_XLSX.exists():
    xlsx_path = STT_KEYWORD_XLSX
    use_domain = False
    print("  키워드: stt.xlsx 사용 (엑셀 부스팅)")
else:
    xlsx_path = None
    use_domain = True
    print("  키워드: 도메인 부스팅만 사용")

# ---- STT 파이프라인 실행 ----
timeline_json_path = run_stt_pipeline(
    audio_path=vocals_path,
    invoke_url=CLOVA_INVOKE_URL,
    secret_key=CLOVA_SECRET_KEY,
    stt_raw_dir=STT_RAW_DIR,
    stt_seg_dir=STT_SEG_DIR,
    xlsx_keywords_path=xlsx_path,
    use_domain_boostings=use_domain,
    speaker_count_min=2,
    speaker_count_max=3,
    save_raw_json=True,
    pause_thresh_ms=50000,  # 필요시 조절
)

print(f"\n✅ STT 완료!")
print(f"timeline.json : {timeline_json_path}\n")

In [ ]:
# ==========================================
# Cell 5: STT 데이터 전처리 (이벤트 세트)
# ==========================================
from src.stt_event_splitter import stt_json_to_event_sets

timeline_json_stem = timeline_json_path.stem
# 같은 디렉터리에 *_output.json 으로 저장
json_after_split_path = timeline_json_path.with_name(f"{timeline_json_stem}_set_split.json")

# 입력 JSON 로드
with timeline_json_path.open("r", encoding="utf-8") as f:
    stt_json = json.load(f)

# 이벤트 세트 변환
event_sets = stt_json_to_event_sets(
    stt_json,
    caster_gap=10.0,   # 필요시 튜닝
    silence_gap=2.0,  # 필요시 튜닝
)

print(f"이벤트 세트 개수: {len(event_sets)}")

# 출력 JSON 저장
with json_after_split_path.open("w", encoding="utf-8") as f:
    json.dump(event_sets, f, ensure_ascii=False, indent=2)

print(f"저장 완료: {json_after_split_path.resolve()}")

In [ ]:
# ==========================================
# Cell 6: 영상 이미지 추출
# ==========================================

import json

from src.image_extraction import capture_frames_for_sets

# 세트 json 로드
with json_after_split_path.open("r", encoding="utf-8") as f:
    sets = json.load(f)   # 리스트 형태여야 함

print(f"세트 개수: {len(sets)}")

# 5) 세트의 set_start_sec 기준으로 이미지 추출
results = capture_frames_for_sets(
    video_path=local_video_path,
    sets=sets,
    output_dir=FRAMES_ROOT
)

# 6) 요약 출력
success_count = sum(1 for r in results if r["success"])
print(f"성공적으로 저장된 이미지 수: {success_count}")


In [ ]:
# ==========================================
# Cell 7: VLM 스코어보드 추출 (최적화 버전)
# ==========================================

from pathlib import Path

from src.vlm_scoreboard import (
    load_scoreboard_model_and_processor,
    attach_scoreboard_to_sets,
)

# ==========================
# 모델 / 프로세서 로드
# ==========================

vlm_model, vlm_processor = load_scoreboard_model_and_processor()


# ==========================
# scoreboard 파이프라인 실행
# ==========================
json_after_split_stem = json_after_split_path.stem
# 같은 디렉터리에 *_output.json 으로 저장
scoreboard_json_path = json_after_split_path.with_name(f"{json_after_split_stem}_scoreboard.json")

updated_sets = attach_scoreboard_to_sets(
    json_after_split_path=json_after_split_path,
    output_json_path=scoreboard_json_path,
    frames_root=FRAMES_ROOT,
    video_path=local_video_path,
    model=vlm_model,
    processor=vlm_processor,
    retry_if_all_null=False,   # all-null이면 +2초 재시도
    retry_offset_sec=2.0,
)

print(f"총 세트 수: {len(updated_sets)}")
print(f"저장 완료: {scoreboard_json_path.resolve()}")

# 일부만 눈으로 확인
for s in updated_sets[:5]:
    print("=" * 80)
    print("set_id:", s["set_id"])
    print("set_start_sec:", s["set_start_sec"])
    print("scoreboard:", s["scoreboard"])

In [ ]:
from pathlib import Path
import json

# from pakchanho_commentary_generator import (
#     load_pakchanho_model,
#     generate_analyst_for_all_sets,
# )

from src.llm_generator import (
    load_pakchanho_model,
    generate_analyst_for_all_sets,
)

# Pakchanho LLM 적용 후 저장할 경로
json_llm_output_path = LLM_OUT_DIR / "vocals_timeline_set_split_scoreboard_pakchanho.json"

# 1) 모델 로드
model, tokenizer = load_pakchanho_model(
    base_model_name="kakaocorp/kanana-1.5-8b-instruct-2505",
    lora_model_id="SeHee8546/kanana-1.5-8b-pakchanho-lora-v2",
    load_in_4bit=True,
)

# 2) 세트별 analyst_text 재생성
game_title = "2025 KBO 준플레이오프 4차전 삼성 vs SSG"  # 경기 정보는 상황에 맞게 바꾸면 됨

result_sets = generate_analyst_for_all_sets(
    json_in_path=scoreboard_json_path,
    json_out_path=json_llm_output_path,
    model=model,
    tokenizer=tokenizer,
    game_title=game_title,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    no_repeat_ngram_size=3,
    base_max_new_tokens=512,
)

print(f"총 세트 수: {len(result_sets)}")
print(f"저장 완료: {json_llm_output_path.resolve()}")

# 3) 샘플 몇 개 확인
for row in result_sets[:5]:
    print("=" * 80)
    print("set_id:", row["set_id"])
    print("caster_text:", row["caster_text"])
    print("analyst_text:", row["analyst_text"])

In [ ]:
# ==========================================
# JSON 기반 TTS 전체 파이프라인 (로컬/RunPod 호환)
# ==========================================

from pathlib import Path
import pandas as pd
import os

from src.json_tts_pipeline import run_full_tts_pipeline_from_json


# ========================================
# 환경 자동 감지 및 경로 설정
# ========================================

def get_project_root() -> Path:
    """
    프로젝트 루트 디렉토리를 자동으로 찾습니다.
    - RunPod: /workspace/skn17_final_runpod_code
    - 로컬: 현재 노트북이 있는 위치 기준으로 탐색
    """
    # 현재 노트북 파일의 디렉토리
    current_dir = Path.cwd()
    
    # RunPod 환경 체크
    runpod_path = Path("/workspace/skn17_final_runpod_code")
    if runpod_path.exists():
        print(f"🔍 환경 감지: RunPod")
        return runpod_path
    
    # 로컬 환경: 상위 디렉토리를 탐색하며 baseball_pipeline 또는 data 폴더 찾기
    for parent in [current_dir] + list(current_dir.parents):
        # baseball_pipeline 폴더가 있는지 확인
        if (parent / "baseball_pipeline").exists():
            print(f"🔍 환경 감지: 로컬 (baseball_pipeline 발견)")
            return parent
        # data 폴더가 있는지 확인
        if (parent / "data").exists():
            print(f"🔍 환경 감지: 로컬 (data 발견)")
            return parent
    
    # 찾지 못했으면 현재 디렉토리 반환
    print(f"🔍 환경 감지: 현재 디렉토리 사용")
    return current_dir

# 프로젝트 루트 자동 감지
PROJECT_ROOT = get_project_root()
print(f"📁 프로젝트 루트: {PROJECT_ROOT}\n")

# DATA_DIR 설정 (baseball_pipeline/data 또는 data)
if (PROJECT_ROOT / "baseball_pipeline" / "data").exists():
    DATA_DIR = PROJECT_ROOT / "baseball_pipeline" / "data"
elif (PROJECT_ROOT / "data").exists():
    DATA_DIR = PROJECT_ROOT / "data"
else:
    # 없으면 생성
    DATA_DIR = PROJECT_ROOT / "data"
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    print(f"⚠️ data 폴더를 생성했습니다: {DATA_DIR}")

print(f"📁 데이터 디렉토리: {DATA_DIR}\n")


# ========================================
# 🎙️ 해설위원 선택 (1, 2, 3 중 선택)
# ========================================
ANALYST_SELECT = 1  # 1: 박찬호, 2: 이대호, 3: 김광현

# 해설위원 매핑
ANALYST_MAPPING = {
    1: "park",   # 박찬호
    2: "lee",    # 이대호
    3: "kim",    # 김광현
}

ANALYST_NAMES = {
    1: "박찬호",
    2: "이대호",
    3: "김광현",
}

if ANALYST_SELECT not in ANALYST_MAPPING:
    raise ValueError(f"❌ ANALYST_SELECT는 1, 2, 3 중 하나여야 합니다. (현재: {ANALYST_SELECT})")

selected_analyst = ANALYST_MAPPING[ANALYST_SELECT]
selected_analyst_name = ANALYST_NAMES[ANALYST_SELECT]
print(f"🎙️ 선택된 해설위원: {selected_analyst_name} ({selected_analyst}) [번호: {ANALYST_SELECT}]\n")

# ========================================
# 파일 경로 설정
# ========================================

# 입력 파일들 (로컬/RunPod에 맞게 설정)
# 로컬에서는 이 변수들을 먼저 정의해야 합니다
if 'local_video_path' not in locals():
    # 예시: 로컬에서 직접 경로 지정
    local_video_path = DATA_DIR / "input_videos" / "your_video.mp4"
    print(f"⚠️ local_video_path가 정의되지 않아 기본값 사용: {local_video_path}")

if 'json_llm_output_path' not in locals():
    # 예시: 로컬에서 직접 경로 지정
    json_llm_output_path = DATA_DIR / "llm_outputs" / "your_json.json"
    print(f"⚠️ json_llm_output_path가 정의되지 않아 기본값 사용: {json_llm_output_path}")

video_stem = Path(local_video_path).stem

# Fish-Speech API 설정
FISH_API_URL = "http://127.0.0.1:8080/v1/tts"

# 캐스터는 고정
CASTER_REF_WAVS = [DATA_DIR / "tts_refs" / "caster_jung.wav"]

# 해설위원은 선택에 따라 동적으로 설정
ANALYST_REF_WAVS = [DATA_DIR / "tts_refs" / f"analyst_{selected_analyst}.wav"]

# ========================================
# 파일 존재 여부 확인
# ========================================
print("=" * 80)
print("📁 참조 음성 파일 확인")
print("=" * 80)

all_refs_exist = True

# 캐스터 참조 확인
for ref_path in CASTER_REF_WAVS:
    if ref_path.exists():
        size_mb = ref_path.stat().st_size / (1024**2)
        print(f"✅ 캐스터 참조: {ref_path.name} ({size_mb:.2f} MB)")
    else:
        print(f"❌ 캐스터 참조 없음: {ref_path}")
        all_refs_exist = False

# 해설위원 참조 확인
for ref_path in ANALYST_REF_WAVS:
    if ref_path.exists():
        size_mb = ref_path.stat().st_size / (1024**2)
        print(f"✅ 해설위원 참조 ({selected_analyst_name}): {ref_path.name} ({size_mb:.2f} MB)")
    else:
        print(f"❌ 해설위원 참조 없음: {ref_path}")
        all_refs_exist = False

print("=" * 80 + "\n")

if not all_refs_exist:
    print("❌ 필요한 참조 음성 파일 경로:")
    print(f"   {DATA_DIR / 'tts_refs'}")
    print("\n필요한 파일:")
    print("   - caster_jung.wav")
    print("   - analyst_park.wav (박찬호)")
    print("   - analyst_lee.wav (이대호)")
    print("   - analyst_kim.wav (김광현)")
    raise FileNotFoundError("❌ 필요한 참조 음성 파일이 없습니다!")

# 입력 파일 확인
print("=" * 80)
print("📁 입력 파일 확인")
print("=" * 80)

if Path(local_video_path).exists():
    size_mb = Path(local_video_path).stat().st_size / (1024**2)
    print(f"✅ 원본 비디오: {Path(local_video_path).name} ({size_mb:.2f} MB)")
else:
    print(f"❌ 원본 비디오 없음: {local_video_path}")
    raise FileNotFoundError(f"원본 비디오 파일이 없습니다: {local_video_path}")

if Path(json_llm_output_path).exists():
    size_kb = Path(json_llm_output_path).stat().st_size / 1024
    print(f"✅ JSON 세트: {Path(json_llm_output_path).name} ({size_kb:.2f} KB)")
else:
    print(f"❌ JSON 세트 없음: {json_llm_output_path}")
    raise FileNotFoundError(f"JSON 파일이 없습니다: {json_llm_output_path}")

print("=" * 80 + "\n")

# ========================================
# TTS 파이프라인 실행
# ========================================

print("[TTS] JSON 기반 TTS 파이프라인 시작")
print(f"  JSON 세트 파일: {json_llm_output_path}")
print(f"  원본 영상: {local_video_path}")
print(f"  캐스터 참조: {CASTER_REF_WAVS}")
print(f"  해설 참조 ({selected_analyst_name}): {ANALYST_REF_WAVS}")
print()

try:
    final_tts_wav, aligned_csv, tts_csv_with_paths = run_full_tts_pipeline_from_json(
        json_sets_path=json_llm_output_path,
        video_path=local_video_path,
        caster_ref_wavs=CASTER_REF_WAVS,
        analyst_ref_wavs=ANALYST_REF_WAVS,
        fish_api_url=FISH_API_URL,
        # 아래 파라미터들은 기존 셋업 그대로 사용 (필요하면 튜닝 가능)
        min_text_chars=2,
        merge_same_role=True,
        merge_gap_thresh_sec=0.25,
        merge_short_thresh_sec=1.0,
        min_gap_sec=0.02,
        caster_extra_ratio=0.2,
        analyst_extra_ratio=2.0,
        max_analyst_expand_sec=7.0,
        analyst_priority_min_overlap_sec=0.5,
        min_gap_ms=60,
        tail_margin_ms=80,
        caster_max_speedup=1.3,
        analyst_max_speedup=1.8,
    )

    print("\n" + "=" * 80)
    print("✅ JSON → TTS → 정렬 → WSOLA 전체 완료!")
    print("=" * 80)
    print(f"  - TTS CSV(with paths): {tts_csv_with_paths}")
    print(f"  - 정렬 CSV: {aligned_csv}")
    print(f"  - 최종 TTS 타임라인 wav: {final_tts_wav}")
    print("=" * 80 + "\n")

    # 통계 출력
    tts_df = pd.read_csv(tts_csv_with_paths)
    print("📊 TTS 통계:")
    print(f"  - 총 발화 수: {len(tts_df)}")
    print(f"  - TTS 성공: {tts_df['tts_wav_path'].notna().sum()}개")
    print(f"  - TTS 실패: {tts_df['tts_wav_path'].isna().sum()}개")
    
    # 역할별 통계
    if 'role' in tts_df.columns:
        print(f"\n📊 역할별 통계:")
        role_counts = tts_df['role'].value_counts()
        for role, count in role_counts.items():
            print(f"  - {role}: {count}개")
            
    print(f"\n🎙️ 사용된 해설위원: {selected_analyst_name}")

except Exception as e:
    print(f"\n❌ JSON 기반 TTS 파이프라인 실패: {e}")
    import traceback
    traceback.print_exc()
    raise

In [ ]:
# ==========================================
# Cell 12: 최종 영상 인코딩 (초고속 최적화 버전)
# ==========================================
import subprocess
from pathlib import Path
import time

OUTPUT_VIDEO_DIR = DATA_DIR / "output_videos"
OUTPUT_VIDEO_DIR.mkdir(parents=True, exist_ok=True)

TTS_AUDIO_DIR = DATA_DIR / "tts_audio" / video_stem
AUDIO_SEPARATOR_DIR = DATA_DIR / "audio_separator"

final_tts_wav = TTS_AUDIO_DIR / f"{video_stem}.tts_timeline.wav"
no_vocals_path = AUDIO_SEPARATOR_DIR / "no_vocals.wav"

def get_duration(file_path: Path) -> float:
    """파일의 길이를 초 단위로 반환"""
    cmd = [
        'ffprobe', '-v', 'error',
        '-show_entries', 'format=duration',
        '-of', 'default=noprint_wrappers=1:nokey=1',
        str(file_path)
    ]
    result = subprocess.run(cmd, capture_output=True, encoding='utf-8', errors='replace', check=True)
    return float(result.stdout.strip())

def merge_audio_and_encode_video_fast(
    original_video_path: Path,
    tts_vocals_path: Path,
    bg_no_vocals_path: Path,
    output_video_path: Path,
    tts_volume: float = 1.0,
    bg_volume: float = 0.7,
) -> Path:
    """
    초고속 버전: 오디오 먼저 믹싱 후 비디오와 결합
    """
    output_video_path.parent.mkdir(parents=True, exist_ok=True)
    
    # 파일 존재 여부 확인
    print("\n" + "=" * 80)
    print("📁 파일 존재 여부 확인")
    print("=" * 80)
    
    files_to_check = {
        "원본 비디오": original_video_path,
        "TTS 오디오": tts_vocals_path,
        "배경음": bg_no_vocals_path,
    }
    
    for name, path in files_to_check.items():
        if path.exists():
            size_mb = path.stat().st_size / (1024**2)
            print(f"✅ {name}: {path.name} ({size_mb:.2f} MB)")
        else:
            print(f"❌ {name}: {path}")
            raise FileNotFoundError(f"{name} 파일이 존재하지 않습니다: {path}")
    
    print("=" * 80 + "\n")
    
    # 파일 길이 확인
    try:
        video_duration = get_duration(original_video_path)
        tts_duration = get_duration(tts_vocals_path)
        bg_duration = get_duration(bg_no_vocals_path)
        
        print(f"[INFO] 📹 비디오: {video_duration:.2f}초 ({int(video_duration//60)}분 {int(video_duration%60)}초)")
        print(f"[INFO] 🎤 TTS: {tts_duration:.2f}초 ({int(tts_duration//60)}분 {int(tts_duration%60)}초)")
        print(f"[INFO] 🎵 배경음: {bg_duration:.2f}초 ({int(bg_duration//60)}분 {int(bg_duration%60)}초)\n")
            
    except Exception as e:
        print(f"[WARNING] 파일 길이 확인 실패: {e}\n")
    
    # ============================================
    # STEP 1: 오디오만 먼저 빠르게 믹싱 (필터 없이)
    # ============================================
    temp_mixed_audio = output_video_path.parent / f"temp_mixed_{output_video_path.stem}.m4a"
    
    print("[STEP 1/2] 🎵 오디오 믹싱 (초고속 모드)")
    start_step1 = time.time()
    
    # 오디오만 처리하므로 매우 빠름
    audio_cmd = [
        'ffmpeg', '-y',
        '-i', str(tts_vocals_path),
        '-i', str(bg_no_vocals_path),
        '-filter_complex',
        f'[0:a]volume={tts_volume}[a1];'
        f'[1:a]volume={bg_volume}[a2];'
        f'[a1][a2]amix=inputs=2:duration=first:dropout_transition=0[aout]',
        '-map', '[aout]',
        '-c:a', 'aac',
        '-b:a', '192k',
        str(temp_mixed_audio)
    ]
    
    result = subprocess.run(audio_cmd, capture_output=True, encoding='utf-8', errors='replace')
    
    if result.returncode != 0:
        print(f"❌ 오디오 믹싱 실패: {result.stderr}")
        raise subprocess.CalledProcessError(result.returncode, audio_cmd)
    
    elapsed_step1 = time.time() - start_step1
    print(f"          ✅ 완료 ({elapsed_step1:.1f}초)\n")
    
    # ============================================
    # STEP 2: 비디오와 오디오 결합 (둘 다 copy)
    # ============================================
    print("[STEP 2/2] 🎬 비디오 + 오디오 결합 (초고속 모드)")
    start_step2 = time.time()
    
    # 비디오와 오디오 모두 복사만 하므로 초고속
    video_cmd = [
        'ffmpeg', '-y',
        '-i', str(original_video_path),
        '-i', str(temp_mixed_audio),
        '-map', '0:v:0',      # 비디오 스트림
        '-map', '1:a:0',      # 오디오 스트림
        '-c:v', 'copy',       # 비디오 복사 (재인코딩 없음)
        '-c:a', 'copy',       # 오디오 복사 (재인코딩 없음)
        '-shortest',
        str(output_video_path)
    ]
    
    result = subprocess.run(video_cmd, capture_output=True, encoding='utf-8', errors='replace')
    
    if result.returncode != 0:
        print(f"❌ 비디오 결합 실패: {result.stderr}")
        raise subprocess.CalledProcessError(result.returncode, video_cmd)
    
    elapsed_step2 = time.time() - start_step2
    print(f"          ✅ 완료 ({elapsed_step2:.1f}초)\n")
    
    # 임시 파일 삭제
    try:
        temp_mixed_audio.unlink()
        print(f"[CLEANUP] 🗑️  임시 파일 삭제\n")
    except:
        pass
    
    total_time = elapsed_step1 + elapsed_step2
    print(f"[TOTAL] ⚡ 전체 소요 시간: {total_time:.1f}초")
    
    return output_video_path

# 최종 출력 영상 경로
final_video_path = OUTPUT_VIDEO_DIR / f"{video_stem}.final.mp4"

print("\n" + "=" * 80)
print("🎬 최종 영상 생성 시작 (초고속 모드)")
print("=" * 80)
print(f"  📹 원본 비디오: {local_video_path.name}")
print(f"  🎤 TTS 음성: {final_tts_wav.name}")
print(f"  🎵 배경음: {no_vocals_path.name}")
print(f"  💾 출력: {final_video_path.name}")
print("=" * 80)

try:
    final_video = merge_audio_and_encode_video_fast(
        original_video_path=local_video_path,
        tts_vocals_path=final_tts_wav,
        bg_no_vocals_path=no_vocals_path,
        output_video_path=final_video_path,
        tts_volume=1.0,
        bg_volume=0.7,
    )
    
    print("\n" + "="*80)
    print("🎉 전체 파이프라인 완료!")
    print("="*80)
    print(f"✅ 최종 영상: {final_video}")
    print(f"📦 파일 크기: {final_video.stat().st_size / (1024**2):.2f} MB")
    
    # 최종 영상 길이 확인
    try:
        final_duration = get_duration(final_video)
        print(f"⏱️  영상 길이: {final_duration:.2f}초 ({int(final_duration//60)}분 {int(final_duration%60)}초)")
    except:
        pass
    
    print("="*80 + "\n")
    
except subprocess.CalledProcessError as e:
    print(f"\n❌ 인코딩 실패!")
    print(f"에러 코드: {e.returncode}")
    raise
    
except Exception as e:
    print(f"\n❌ 실패: {e}")
    import traceback
    traceback.print_exc()
    raise